# Lab 5 - Debugging Neural Networks

In this notebook we'll take a look at three common issues encountered when building and training neural networks. As a lot of the code has been written for you, work slowly through the notebook, reading along the way - there is some important information!

## Data Normalisation

In this exercise we'll take a look at a form of [normalisation](https://en.wikipedia.org/wiki/Normalization_(statistics)) and how it can aide the training of neural networks (and other optimisation techniques, for that matter). As mentioned in the lectures, neural networks are most stable when the inputs and outputs have values near zero, with a standard deviation of approximately 1. Let's see what this looks like in practice.

We will demonstrate the importance of normalisation using an interesting forecast prediction dataset. In this dataset we will use cartographic variables of small regions of forest (30m x 30m) to predict the type of forecast cover in each region. Example cover types include: spruce/fir (type 1), lodgepole pine (type 2), etc.

You can get more information about the data from this link.
https://archive.ics.uci.edu/ml/datasets/covertype

There is also a kaggle competition for this dataset:
https://www.kaggle.com/c/forest-cover-type-prediction/overview

According to the kaggle competition the best team got an accuracy of 100% on the test set! 


In [ ]:
pip install torchmetrics

In [ ]:
# Quickly set up our device
import torch
import torch.nn as nn
import numpy as np
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
print("Training on", device)

Here is the code we use to load the data into a pandas dataframe.

In [ ]:
import pandas as pd

# Read the data from csv file
cover_type_df = pd.read_csv('https://github.com/zhenhe1/datasets/raw/5852941fa9cd21326d3f7277d971e46e48ea2714/covtype_dataset.zip')


Lets take a look at the data using the next cell. Each row of the data contains the data for one small 30m x 30m region of the forest. You will notice the cartographic variables include things like elevation, aspect, slope, soil types etc. You will notice that the soil type values are much smaller than elevation, aspect, hydrology measurements. This data will need to be normalised, ensuring all input features are **approximately** in the range `[-1,1]`. There are two reasons for normalisation:

1. If the average magnitude of one feature is higher than another, then that feature is implicitly more important during early epochs (the model might eventually learn around this and give that feature a lower weight). By normalising, you ensure that each feature is treated equally at all times and the model can choose which features are important.
2. Loss functions and default learning rates are designed assuming your inputs and outputs are normalised.


We will first try to train a network without normalisation and then, later, add normalisation and see what happens.


In [ ]:
cover_type_df

<a name='create-dataset'></a>
### Create a dataset class
In the next cell we will create a dataset class for our forest cover data. Notice this time the dataset class will directly take a pandas dataframe and then extract the input and labels from it. The data has 55 columns, the first 54 columns contain the cartographic variables used to predict forecast cover type. Notice we convert the data into numpy arrays instead of PyTorch tensors. It is fine for datasets to output numpy arrays since we will wrap a dataloader around the dataset, and the dataloader will automatically convert the numpy arrays into PyTorch tensors.

<font color=red>Complete the `CovDataset` class below, except for the last `TODO`. This will be completed later.</font>

In [ ]:
# Import the base class
from torch.utils.data import Dataset

# Subclassing in Python just requires adding the class name in parentheses
class CovDataset(Dataset):
    # The __init__ method is similar to a constructor like you find in other
    # languages. 
    def __init__(self, df):
        # We directly take the pandas data frame split the data into the inputs
        # and labels.
        # The first 54 columns contain the input attributes
        # The last column called Cover_type contain the data we want to predict
        self.inputs = df[df.columns[:54]].to_numpy().astype(np.float32)
        self.labels = df.Cover_Type.to_numpy().astype(np.int64)
        # We make a deep copy of the inputs and store it as a copy of the original.
        # This way when we normalize the data in a future exercise we can make sure
        # we always apply the normalization based on the original inputs.
        self.original_inputs =  self.inputs.copy()

    def __len__(self):
        # TODO: Return the length (number of items) in the dataset


    def __getitem__(self, idx):
        # TODO: Store the inputs and labels for the item at index idx in variables
        #       input and label
        # input = ...
        # label = ...
        
      
        return input, label

    def normalise_data(self, mean, std):
        pass
        # TODO: There will be instructions to complete this function later.
        #       For now, SKIP THIS TODO.
        #       When it is time to implement this method:
        #         1. Implement the Standard score. 
        #            i.e. subtract the mean and divide by the std.
        #         2. Remember to only change self.inputs, 
        #            not self.original_inputs!

      

<a name="create-splits"></a>
### Create train, validation and test dataset splits and dataloaders

Notice the difference between this labs data loading code versus lab2a. The main difference is that we created a dataset class in the previous cell and passed the dataframe into it. So all the messy extraction of input data and labels are done in the dataset class. Writing a dataset class is considered a more elegant way to code dataloading. So we encourage you to do that in the future as well.

You will notice the this dataset is quite large.

<font color=red>In the next cell complete the code to create a Dataset object, split it into train/validation/test and finally create a DataLoader for each Dataset.</font>



In [ ]:
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split

# The percentages for each partition
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1
TEST_SPLIT = 0.1
BATCH_SIZE = 256

# Ensure that the splits add to 100%
assert TRAIN_SPLIT + VAL_SPLIT + TEST_SPLIT == 1

# TODO : create the forecast cover dataset (CovDataset) object using the cover_type_df
#        dataframe.
# entireCoverDataset = 


# Calculate the number of examples in each partition
train_size = int(TRAIN_SPLIT * len(entireCoverDataset))
val_size = int(VAL_SPLIT * len(entireCoverDataset))
test_size = len(entireCoverDataset) - train_size - val_size

print("Train examples:     ", train_size)
print("Validation examples:", val_size)
print("Test examples:      ", test_size)


#TODO Just like you did in lab2a please use the random_split function to 
#     break entireCoverDataset into train_dataset, val_dataset, test_dataset
# train_dataset, val_dataset, test_dataset = 




#TODO Just like you did in lab2a please create the train_loader, val_loader
#     and test_loader using their corresponding datasets.
#     Please do random shuffling for the train_loader, do not do random shuffle
#     for the other two data loaders

# train_loader =
# val_loader =
# test_loader = 


Lets take a look at the first element of the training dataset that we created. You should see a tuple where the first element contains a 1D array with 54 elements and the second element stores the output label.

In [ ]:
print(train_dataset[0])

### MLP model

In the cell below finish the definition of a MLP model that will be used in the training loop. Notice we have already specified the default input and output size for the model. 

<font color=red>In the next cell, add the requested layers to the `nn.Sequential()`</font>

In [ ]:

# Custom models need to subclass nn.Module
class MLP(nn.Module):
    def __init__(self,
                 device,
                 input_size=54, # default to 54
                 output_size=8 # default to the number of classes
                 ):
        super().__init__()

        # Write the classifier layers here.
        self.seq = nn.Sequential(
            
            ## TODO: add the following layers
            #        Linear layer that outputs 64 neurons
            #        ReLU
            #        Linear layer that outputs 64 neurons
            #        ReLU
            #        Linear layer that outputs output_size neurons

        
        )
        # Transfer the model weights to the GPU
        self.to(device)
    
    # The forward method takes input values and returns predictions. We just need
    # to pass the inputs through the layers we defined in __init__
    def forward(self, x):
        return self.seq(x)

### Training loop
To keep things simple we have just copied the training loop used in lab 2b here to do the training. Note we are not using the validation dataset in order to keep the training code short. We have set the optimizer to the Adam optimizer with initial learning rate of 0.001.

Lets train the model and see what happens.

In [ ]:
import torch.optim as optim
from tqdm.notebook import tqdm
import torchmetrics

criterion = nn.CrossEntropyLoss()
model = MLP(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#initialize metric
train_accuracy = torchmetrics.Accuracy(task = 'multiclass', num_classes = 8).to(device)

# The number of times we loop over the entire dataset
total_epochs = 10
for epoch in tqdm(range(total_epochs)): 
    epoch_train_accuracy = []
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)   
        acc = train_accuracy(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print('epoch: %d loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))
    print('epoch: %d accuracy: %.3f' % (epoch + 1, train_accuracy.compute()))
    # reset for the next epoch
    train_accuracy.reset()
print('Finished Training')


If everything goes according to plan you will notice that the training accuracy is around 77%. This is not that high considering other people can get 100% accuracy. Lets see what happens when we try it on the test dataset.

In [ ]:
running_loss = 0.0
test_accuracy = torchmetrics.Accuracy(task = 'multiclass', num_classes = 8).to(device)

for i, data in enumerate(test_loader, 0):
   inputs, labels = data
   inputs, labels = inputs.to(device), labels.to(device)

   model.eval()
   with torch.no_grad():
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
    acc = test_accuracy(outputs, labels) 

print("test loss: ", running_loss/len(test_loader))
print('test accuracy: ', test_accuracy.compute())
# reset internal state to make the metric ready for new data
test_accuracy.reset()

### Normalising the data
The test dataset also just reports accuracy of around 77%. Lets see if using normalisation can significantly improve the results.

There are several forms of normalisation. The normalisation we will do is called the [Standard score](https://en.wikipedia.org/wiki/Standard_score#Calculation) or z-score. To calculate the Standard score, you subtract the mean and divide by the standard deviation. So this means we need to compute the mean and standard deviation of every column of the data resulting in **54 means** and **54 standard deviations**. 

Doing this normalisation to each column will make the values in each column have similar range of values and make them small and centered around zero. This will improve training by ensuring the inputs have similar magnitudes and thus are weighted equally, and will better match the default learning rate of Adam.

The purpose of a validation/testing set is to test generalisation: i.e. to check how well the model does on unseen examples. You must never use any part of the testing set to tune your model, else you are no longer correctly testing generalisation. Thus, we will calculate the mean and standard deviation from only the training set and then apply it to all splits equally. If we were to deploy a model, then we would use the mean/std from the training set to normalise the inputs there, too.

<font color=red>In the next code cell extract the numpy array containing the input data from the **training** dataset and calculate the means/stds of **all** columns.</font>






In [ ]:
# Here we extract the input data of the training dataset into a
# numpy array called train_inputs.
train_inputs = entireCoverDataset.inputs[train_dataset.indices]

# TODO : write code to extract the mean and standard deviation for each column 
#        of the data and store them in the following variables.
#        train_inputs_means = 
#        train_inputs_stds = 


assert train_inputs_means.shape == (54,)
assert train_inputs_stds.shape == (54,)

We will now use these means and standard deviations (stds) to normalise our data. 

<font color=red>

1. Go back to `CovDataset` in [Create a dataset class](#create-dataset) and add a function called `normalise_data`.
2. Re-run the cell in the [Create train, validation and test dataset splits and dataloaders](#create-splits).
3. Run the next cell to call the `normalise_data` function you added to the CovDataset object.

</font>

*Note: The `random_split` function does not copy the data into the train/validation/test datasets. Instead, they are views into the `entireCoverDataset`. Thus we only need to normalise `entireCoverDataset`.*

Once you have normalised the data, please go back to the training loop and execute it again. See how much higher the training and test accuracy goes. You should see a very significant increase in the accuracy (around 10% higher accuracy).

If you have time at the end of the lab. Feel free to come up with better models and also try different optimisation parameter values to see if you can push the accuracy even higher.

In [ ]:
# We call the normalise_data function on the entire data set.
# This normalises all the input data for the train, validation and test splits
# in one go. 
# TODO: Please go to the CovDataset class and add a function called
#       normalize_data which normalises the input by subtracting the mean
#       and dividing by the standard deviation from the original_input.
entireCoverDataset.normalise_data(train_inputs_means, train_inputs_stds)

## Learning Rate and Class Imbalance

In the past, we've mentioned the effects of a poor choice of learning rate, and we'll now look at a practical example.

To demonstrate this, we'll build a simple image dataset consisting of white squares and rectangles on a black background, with the task of classifying them as either squares or rectangles.

### Preparation

The dataset has been implemented you. Feel free to read through it - although the implementation isn't particularly important.

Each example in the dataset consists of a white rectangle of a randomly chosen width and height, with a label of `1` if it's a square and `0` if it's a rectangle.

In [ ]:
from torch.utils.data import Dataset

class ToyImageDataset(Dataset):
    def __init__(self, num_examples):
        super().__init__()
        height, width = 12, 12
        self.images, self.labels = self.gen_examples(num_examples, height, width)

    def gen_examples(self, num_examples, height, width):
        # Rectangles can be at most half the width or height of the image
        max_height, max_width = height // 2, width // 2

        # Initialise a tensor of zeros to fill in with rectangles
        images = torch.zeros((num_examples, 1, height, width), dtype=torch.float32)
        #images = torch.zeros((num_examples, 1, height, width), dtype=torch.int32)

        # Randomly generate the top-left corner of each rectangle
        tops = torch.randint(0, height - 1, (num_examples,))
        lefts = torch.randint(0, width - 1, (num_examples,))
        # Randomly generate the heights and widths
        heights = torch.randint(1, max_height + 1, (num_examples,))
        widths = torch.randint(1, max_width + 1, (num_examples,))
        # Calculate the bottom-right corner of each rectangle, being sure to not
        # go beyond the image edge
        bottoms = torch.clamp(tops + heights, 0, height)
        rights = torch.clamp(lefts + widths, 0, width)

        # Colour in the rectangles
        for i, (t, l, b, r) in enumerate(zip(tops, lefts, bottoms, rights)):
            images[i, :, t:b, l:r] = 1

        # Squares are rectangles with equal widths and heights
        labels = (heights == widths).type(torch.float32).unsqueeze(-1)
        #labels = (heights == widths).type(torch.int32).unsqueeze(-1)
        return images, labels

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

Run the next cell to create the datasets and dataloaders.

In [ ]:
from torch.utils.data import DataLoader
torch.manual_seed(42)

BATCH_SIZE = 64

train_dataset = ToyImageDataset(900)
test_dataset = ToyImageDataset(100)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Let's take a look at some examples from the training dataset. You'll notice that there are very few square examples - we'll return to this later.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
for i, (image, label) in enumerate(train_dataset):
    if i == 24: break
    plt.subplot(4, 6, i+1)
    plt.imshow(image[0], cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.title('Square' if label else 'Rect')
plt.show()

The CNN for this task is implemented below - note that as we're doing *binary* classification (between two classes), we output a single value. A value less than `0.5` is a prediction of class `0` and a value greater than `0.5` is a predictions of class `1`.

In [ ]:
import torch.nn as nn

# This Printer class is just used to print out the shape of the tensor before
# putting it into the linear layer so we know what is the input size for the linear
# layer.
class Printer(nn.Module):
    def forward(self, x):
        print(x.shape)
        return x

class ConvNet(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(1, 2, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(2, 4, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(4, 4, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(4, 4, 3, padding=1),
            nn.ReLU(),
            # Printer(),
            nn.Flatten(),
            nn.Linear(36, 1)
        )
        self.to(device)

    def forward(self, x):
        return self.seq(x)

This function plots training and testing values (we'll use it to graph the loss and accuracy).

In [ ]:
def plot(title, train_vals, test_vals):
    plt.plot(range(len(train_vals)), train_vals, label='Train')
    plt.plot(range(len(test_vals)), test_vals, label='Test')
    plt.legend()
    plt.title(title)
    plt.show()

Here we have a function to perform training and testing.

In [ ]:
from tqdm.notebook import tqdm

def train_and_test(model, train_loader, test_loader, loss_func, optimiser, num_epochs, accuracy=None):
    train_losses, test_losses = [], []
    train_accs, test_accs = [], []
    for epoch in tqdm(range(num_epochs)):
        sum_loss, sum_acc = 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimiser.zero_grad()
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimiser.step()
            sum_loss += loss.item()
            if accuracy: acc = accuracy(outputs, labels.type(torch.int32))
        train_losses.append(sum_loss / len(train_loader))
        train_accs.append(accuracy.compute().cpu())
        if accuracy: accuracy.reset()

        sum_loss, sum_acc = 0, 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            sum_loss += loss.item()
            aaa = labels.type(torch.int32)
            if accuracy: acc = accuracy(outputs, labels.type(torch.int32))
        test_losses.append(sum_loss / len(test_loader))
        test_accs.append(accuracy.compute().cpu())
        if accuracy: accuracy.reset()
    return train_losses, test_losses, train_accs, test_accs

<a name="train-model"></a>
### Train the Model

We now call the training and plotting functions to train and view the training and testing accuracy curves.

In [ ]:
import torchmetrics

torch.manual_seed(42)
model = ConvNet(device)
# When predicting a probability (e.g. probability that the image is of a square)
# it's relatively common for the last layer of the network to be nn.Sigmoid(),
# as this ensures the outputs are between 0 and 1.
# However, Binary Cross Entropy Loss expects the input to be logits, which means
# we should not use nn.Sigmoid(), as this loss function already does that. 
loss_func = nn.BCEWithLogitsLoss()



# TODO: First try the code. Then adjust the learning rate (lr) to something more reasonable like 1e-3.
optimiser = torch.optim.Adam(model.parameters(), lr=1)

num_epochs= 1000 

accuracy=torchmetrics.Accuracy(task = 'binary', threshold=0.5).to(device)

train_losses, test_losses, train_accs, test_accs = train_and_test(model,
                                                  train_loader,
                                                  test_loader,
                                                  loss_func,
                                                  optimiser,
                                                  num_epochs,
                                                  accuracy)

plot("Accuracy", train_accs, test_accs)

print("train loss: ", train_losses[-1])
print("train accuracy: ", train_accs[-1])
print("test loss: ", test_losses[-1])
print("test accuracy: ", test_accs[-1])


### Fix the Learning Rate Problem

That train acurracy is very unstable! Also the test accuracy just stays flat  This sort of accuracy curve is usually indicative of a badly-selected learning rate. The learning rate above is set to `1`, which would be considered a very high learning rate.


<font color='red'>Adjust the learning rate and train the model again. Try something smaller like 1e-3.</font>

### Detecting the class imbalance problem

The test accuracy looks weird, it starts at 80% and then only moves to around 85%. Lets now look a bit further under the hood. By looking at the distribution of rectangle and square examples in the train and test sets.

In [ ]:
train_square_proportion = len(torch.nonzero(train_dataset.labels)) / len(train_dataset.labels)
train_rect_proportion = 1 - train_square_proportion

test_square_proportion = len(torch.nonzero(test_dataset.labels)) / len(test_dataset.labels)
test_rect_proportion = 1 - test_square_proportion

print("Train data - Square: ", len(torch.nonzero(train_dataset.labels)))
print("Train data - Rectangle: ", len(train_dataset.labels)- len(torch.nonzero(train_dataset.labels)))
print("Train data - Rectangle proportion:", "{:.3f}".format(train_rect_proportion), "Square proportion:", "{:.3f}".format(1- train_rect_proportion))
print("Test data - Rectangle proportion:", "{:.3f}".format(test_rect_proportion), "Square proportion:", "{:.3f}".format(1- test_rect_proportion))


It turns out the reason the model initially got 80% accuracy was because it just always predicted rectangle no matter what input it got. Since there was 80% rectangles in the test set it got the answer correct 80% of the time. Later the model was able to sometimes predict Squares but still not often enough hence it did not move much higher than 80%. There are so many more examples of rectangles compared to squares so during backprop most of the gradients try to steer the model towards predicting rectangles. 

Lets take a look at the confusion matrix on the trained model to confirm our intuition that the model is predicting too many rectangles instead of squares.

The first cell below creates a function that will plot a confusion matrix with some defaults.

The second cell contains the code that uses the `plot_confusion_matrix` function defined in the first cell to draw our confusion matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay

def plot_confusion_matrix(cm, classes, normalise=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalise:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalised confusion matrix")
    else:
        print('Confusion matrix, without normalisation')
    
    fig, ax = plt.subplots()
    cmd = ConfusionMatrixDisplay(cm, display_labels=classes)
    fmt = '.2f' if normalise else 'd'
    cmd.plot(cmap=plt.cm.Blues, ax=ax, values_format=fmt)
    ax.set_title(title)

Our code to draw the confusion matrix of our model results

In [ ]:
from torchmetrics import ConfusionMatrix

# Initialize metric
metric = ConfusionMatrix(task = 'binary').to(device)

for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    metric(outputs, labels.type(torch.int32))

cm = metric.compute().detach().cpu().numpy()
plot_confusion_matrix(cm, ["rectangle", "square"])

The bottom left and top right cells are where the predictions are different from the true labels. This result looks not too bad. It seems we are making a bit more mistakes predictions when the ground truth is squares (bottom left cell) than when the ground truth is rectangles (top right cell). But the problem looks much more obvious when we look at the next cell where we plot the normalised confusion matrix. 

In [ ]:
plot_confusion_matrix(cm, ["rectangle", "square"], normalise=True)

**This result shows us a really big problem!** Look at the bottom left cell. It says that we wrongly predicted about 55% (you may get something between 45% to 70%) of the images that contained squares. So this really shows us our result is actually much worse than the around 85% accuracy reported earlier. We need to use a different metric that better reflects our true performance in terms of both predicting rectangles and squares. So a fairer metric that treats rectangles and squares equally.

So lets turn to the unweighted average recall (UAR) metric which we discussed in lecture 4 to help us out. The UAR metric essentially assigns equal importance to correctly predicting rectangles and squares when they are present in the image. 

The next cell computes the UAR performance of our model on the test set.




In [ ]:
import numpy as np
from torchmetrics import Recall

#initialize metric
# We are using multiclass recall rather than binary recall since binary recall in TorchMetrics
# does not support UAR and also does not allow us to print the recall for each class separately.
recall_perclass = Recall(task = 'multiclass', num_classes = 2, average='none').to(device)
recall_uar = Recall(task='multiclass', num_classes = 2, average='macro',).to(device)
# Gather targets and predictions for the whole test set
targets, predictions = [], []
for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    # Note this code is used to convert the binary outputs to a separate predicted probability
    # for rectangles and squares since the multiclass recall requires separate outputs per class
    # so the shape goes from (64, 1) to (64, 2)
    # Normally this line is not needed if your model already outputs multiple classes probabilities
    outputs = torch.cat((1.0- outputs, outputs), 1)
    labels = labels.squeeze(1)
    recall = recall_perclass(outputs, labels.type(torch.int32))
    uar = recall_uar(outputs, labels.type(torch.int32))

recall_rects, recall_squares =  recall_perclass.compute().detach().cpu().numpy()
print("recall for rectangles =", recall_rects)
print("   recall for squares =", recall_squares)
print("                  UAR =", recall_uar.compute().detach().cpu().numpy())

Notice the recall for squares is much lower than the recall for the rectangles. The UAR (the average of the two recalls) is about 0.7 (the number you get maybe different should be mostly similar). Lets try some techniques to help balance the training.

### Fix the Class Imbalance Problem: WeightedRandomSampler

Now that we have a method for truly measuring the model's performance, we should fix the class imbalance.

The simplest technique for correcting this is to change the proportion of examples that we present to the model. If we randomly select examples without any consideration for the imbalance, we will show the model 80% rectangles and 20% squares on average - we would like to instead show it 50% of each.

PyTorch provides a class [WeightedRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler) to help us with this. Instead of giving each example the same chance of being selected from the dataset, we can weight their selection probability - so an example with a weight of `2` is twice as likely to be selected as an example with a weight of `1`.

To undo the imbalance, we choose the weights based on the proportion of examples. So if classes `A` and `B` have a 9:1 ratio in a dataset, we would need to weight class `A` by a factor of 1, and class `B` by a factor of 9.

To compute this directly, the weights of a class should be $1 - p$, where $p$ is the proportion of that class in the dataset. We have already computed the proportions for each of our two classes, so we can use this equation to compute their appropriate weights.

<font color='red'>In the next cell, compute the sampling weights for the two classes.</font>

In [ ]:
# TODO: Compute the square and rectangle weights using the above equation
# square_weight = ...
# rect_weight = ...


The [WeightedRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler) class requires a weight for each example in the dataset, so we will iterate over every label in the training dataset and build a list of the correct weight. Run the next cell to do so.

In [ ]:
weights = [square_weight if l == 1 else rect_weight for l in train_dataset.labels]

# Print out the first 5 weights
print(weights[:5])

All that's left now is to initialise the sampler and use it to create a new train dataloader.

In the next cell, initialise a `torch.utils.data.sampler.WeightedRandomSampler` by providing the weights list and the length of the train dataset as arguments.

If your code is correct then the number of squares should be roughly half of the number of examples in the training set.

In [ ]:
# TODO: Initialise a WeightedRandomSampler
# sampler = ...


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)

total = 0
for inputs, labels in train_loader:
  total = total + labels.detach().cpu().numpy().sum()

print("number of squares: ", total)
print("total number of training examples:", len(train_dataset))


Once you've completed the above cell, return to the [Train the Model](#train-model) section to try it out.

Once training has completed, run the cell that computes the UAR. If all goes well, the square recall should and the UAR should be higher than before as well. This is because we are showing the model balanced porportions of squares and rectangles throughout training. However, don't be too stressed if the square recall and UAR does not go higher. We have noticed there is some randomness in the results due to the small number of different square examples used during training. You may need to run again to see the benefits. The majority of the times when you rerun the training with the balanced sampler the results should be better. But you might get unlucky once or twice.

Although this method is very useful for handling class imbalance, it can't perform magic - there is a limit to its efficacy. So your UAR will still not be that high. There are other ways to handle skewed distributions. 



### Fix the Class Imbalance Problem: weighted loss function

There is another way to address skewed training distributions. We can increase the size of the gradients going back for training examples of the minority class. This way we can train in a more balanced way without the need to adjust the sampling rate of the two classes. 

The loss that we are currently using is called `torch.nn.BCEWithLogitsLoss`. It turns out that it already has a parameter called`pos_weight` that we can use to adjust the weighting we assign to the positive class (the class with label 1).

Take a look at this documentation to work out how you can modify the loss so the minor class gets more weight :
https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html

It is up to you to decide what weight to use and how to code it. Good luck! You should be able to get better results than using the balanced sampling method.

<font color=red>

1. Run the next cell to reset the dataloader so that it doesn't use WeightedRandomSampler
2. Determine what weights to use and modify the cell in [Train the Model](#train-model) accordingly

</font>


In [ ]:
# Run this code to reset the dataloader so we do not use the weighted sampler
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Challenge

As a challenge try to change the model, learning rate, optimization algorithms, increase the number of training epochs, play around with the loss weight, etc. to see if you can improve the UAR score, and let us know the best UAR you can get! Note that due to the small number of square training examples, you might find a lot of random fluctuation in your test accuracy which makes optimization very hard, so don't spend too much time on this. You should keep a record of what you tried in an excel spreadsheet like we did in lab 3. This is a really important habit to form since people tend to forget what they tried after some time.

You can also play around and see if you can optimize the accuracy of the forest cover classification problem as well. This is a much larger dataset and so produces more stable results. Also for fun you might want to see if the class distribution of this dataset is also imbalanced. If so, you want to do something about that too!

Good luck! By the end of this lab you will have practiced optimizing many different neural network models across the last few labs! You are starting to turn into a neural network optimizer pro!

That's it for this lab - nice work! Hopefully you've learnt something about debugging neural networks. Although there are many ways in which a neural network can fail, we've now seen three of the most common problems.